In [1]:
import os
import OpenEXR
import Imath
import imageio

import torch
import numpy as np

import nvdiffrast.torch as dr

from render import render, util, mesh, light
from render.renderutils.glint_brdf.glint_utils import PCG3dFloat

In [2]:
USE_GLINT = True
itr = 0 # Go to check the getMV function to see how this determines the angle of rendering

if USE_GLINT:
    _ScreenSpaceScale = 0.001
    _LogMicrofacetDensity = 50.0
    _DensityRandomization = 10.0

In [3]:
# Input
data_root_dir = os.path.join(os.getcwd(), 'data')
ref_mesh_path = os.path.join(data_root_dir, 'meshes', 'bob.obj')
ref_mesh_mat_path = os.path.join(data_root_dir, 'meshes', 'bob_glint.mtl') if USE_GLINT else os.path.join(data_root_dir, 'meshes', 'bob_metallic.mtl')
envlight_path = os.path.join(data_root_dir, 'irrmaps', 'clarens.hdr')
# Output
output_root_dir = os.path.join(os.getcwd(), 'out', 'rendered_imgs')
output_file_name = f"out_itr{itr}_{_ScreenSpaceScale}_{_LogMicrofacetDensity}_{_DensityRandomization}" if USE_GLINT else f"out_metallic_itr{itr}" 
output_file_name = f'{output_file_name}_gumbel_softmax'
output_file_name = 'out'

In [4]:
output_file_name

'out'

In [5]:
# Initialise rasteriser
glctx = dr.RasterizeGLContext()
ref_mesh = mesh.load_mesh(ref_mesh_path, ref_mesh_mat_path)
ref_mesh = mesh.compute_tangents(ref_mesh)
envlight = light.load_env(envlight_path)

if USE_GLINT:
    pcg3d = PCG3dFloat(3,3)
    pcg3d.load_state_dict(torch.load(os.path.join('render', 'renderutils', 'glint_brdf', 'weights', 'model_state_dict.pth')))

    ref_mesh.material['glint_params'] = torch.tensor([_LogMicrofacetDensity, _DensityRandomization, _ScreenSpaceScale], device='cuda') # Comment out this line to use regular rendering without glints
    ref_mesh.material['glint_pcg3d'] = pcg3d.to('cuda')
    # ref_mesh.material['glint_4d_noise'] = torch.rand(800, 800, 4, device='cuda')
    ref_mesh.material['glint_4d_noise'] = torch.load(os.path.join(os.getcwd(), 'render', 'renderutils', 'glint_brdf', 'noise_maps', 'glint_4d_noise_800.pt')).to('cuda')

Using /home/w123/.cache/torch_extensions/py39_cu116 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/w123/.cache/torch_extensions/py39_cu116/renderutils_plugin/build.ninja...
Building extension module renderutils_plugin...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module renderutils_plugin...


In [6]:
fovy = np.deg2rad(45)
display_res = [512, 512]
cam_near_far = [0.1, 1000.0]
cam_radius = 3.0
spp = 1
layers = 1

In [7]:
def getMV(itr):
    # Smooth rotation for display.
    ang = (itr / 50) * np.pi * 2
    return util.translate(0, 0, -cam_radius) @ (util.rotate_x(-0.4) @ util.rotate_y(ang))
    
def getMVP(itr):
    proj_mtx = util.perspective(fovy, display_res[1] / display_res[0], cam_near_far[0], cam_near_far[1])
    
    return (proj_mtx @ getMV(itr))[None, ...].cuda()

def getCamPos(itr):
    return (torch.linalg.inv(getMV(itr))[:3, 3])[None, ...].cuda()

def save_as_exr(img, path):
    R = img[..., 0].tobytes()
    G = img[..., 1].tobytes()
    B = img[..., 2].tobytes()
    
    # Create an EXR file
    HEADER = OpenEXR.Header(img.shape[1], img.shape[0])
    exr = OpenEXR.OutputFile(path, HEADER)
    exr.writePixels({'R': R, 'G': G, 'B': B})
    exr.close()

def exr_to_png(input_exr, output_png, use_glsl=False):  
    # Open the EXR file
    exr_file = OpenEXR.InputFile(input_exr)
    
    # Get the size of the image
    dw = exr_file.header()['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1

    # Define the channel type and extract R, G, B channels
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    R = np.frombuffer(exr_file.channel('R', pt), dtype=np.float32).reshape(height, width)
    G = np.frombuffer(exr_file.channel('G', pt), dtype=np.float32).reshape(height, width)
    B = np.frombuffer(exr_file.channel('B', pt), dtype=np.float32).reshape(height, width)

    # Stack the channels to get an RGB image
    img = np.stack([R, G, B], axis=-1)

    if use_glsl:
        ## Adjusting the image to save as PNG
        # Apply Reinhard tone mapping: c' = c / (c + 1)
        img_reinhard = img / (img + 1.0)
        # Apply gamma correction: c' = c^(1/gamma)
        img_corrected = np.power(img_reinhard, 1.0 / 2.2)
    else: # This one's output will look the same as the EXR in tev
        # Convert NumPy array to PyTorch tensor
        img_tensor = torch.from_numpy(img)
        # Apply sRGB to linear RGB conversion
        img_corrected = util.rgb_to_srgb(img_tensor)
        # Convert the corrected tensor back to NumPy array
        img_corrected = img_corrected.numpy()
    
    # Clip and convert the image data to the range [0, 1] and then to uint8
    img_clipped = np.clip(img_corrected, 0, 1)
    img_8bit = (img_clipped * 255).astype(np.uint8)
    # Save the image as PNG
    imageio.imwrite(output_png, img_8bit)

In [8]:
img = render.render_mesh(glctx, ref_mesh, getMVP(itr), getCamPos(itr), envlight, display_res, spp=spp, 
                                num_layers=layers, msaa=True, background=None)['shaded']
img_detached = img.detach().cpu().numpy()[0]

training glints


In [9]:
# View the image in Tev after saving
output_exr_path = os.path.join(output_root_dir, f'{output_file_name}.exr')
save_as_exr(img_detached, output_exr_path)

In [10]:
# Example usage
exr_to_png(output_exr_path, os.path.join(output_root_dir, f'{output_file_name}_nvdiffrec.png'))
exr_to_png(output_exr_path, os.path.join(output_root_dir, f'{output_file_name}_glsl.png'), True)